## 爬取B站音频，在线语音识别成文字，然后GPT总结要点

In [2]:
%pip install requests
%pip install pydub
%pip install openai
%pip install langchain
%pip install llama_index
!pip install spacy && python -m spacy download zh_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/48.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/48.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/48.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/48.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/48.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/48.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/48.5 MB ? eta -:--:--
     --------------------------------------- 0.0/48.5 MB 131.3 kB/s eta 0:06:10
     --------------------------------------- 0.0/48.5 MB 187.9 kB/s eta 0:04:19
     -

In [2]:
# 爬取B站音频

import requests
import re
import json
import subprocess  
import os

# 要请求的网址：B站视频网址
url = "https://www.bilibili.com/video/BV1Z4411179m/?spm_id_from=333.999.0.0&vd_source=56846a54d0076a63db9becb9c940d0b0"

# 添加headers请求头，对Python解释器进行伪装
# referer 和 User-Agent要改写成字典形式
headers = {
    "referer":"https://www.bilibili.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
}

# 用 requests 的 get 方法访问网页
response = requests.get(url=url, headers=headers)

# 返回响应状态码：<Response [200]>
print("返回200，则网页请求成功：",response)

# .text获取网页源代码
# print(response.text)

# 提取视频标题
# 调用 re 的 findall 方法，去response.text中匹配我们要的标题
# 正则表达式提取的数据返回的是一个列表，用[0]从列表中取值
title = re.findall('<h1 title="(.*?)"', response.text)[0]
# 如果标题里有[\/:*?<>|]特殊字符，直接删除
title = re.sub(r"[\/:*?<>|]","",title)
print("视频标题为：",title)


# type函数查看title的数据类型
# print(type(title))

# 提取 playinfo 里的数据
# 调用 re的 findall 方法，去 response.text 中匹配我们要的数据
# 正则表达式提取的数据返回的是一个列表，用[0]从列表中取值
html_data =  re.findall('<script>window.__playinfo__=(.*?)</script>', response.text)[0]

# html_data是字符串类型，将字符串转换成字典
json_data=json.loads(html_data)

# 让pycharm控制台以json格式化输出
# 不影响程序，只改变pycharm或vscode编辑器的终端输出显示
# indent=4 缩进4个空格
json_dicts = json.dumps(json_data,indent=4)

# print(json_dicts)

# # 提取视频画面网址
# video_url = json_data["data"]["dash"]["video"][0]["baseUrl"]
# print("视频画面地址为：", video_url)
# 提取音频网址
audio_url = json_data["data"]["dash"]["audio"][0]["baseUrl"]
print("音频地址为：", audio_url)

# response.content获取响应体的二进制数据
audio_content = requests.get(url=audio_url,headers=headers).content

# 创建mp3文件，写入二进制数据
audio_tmp_file = "./data/" + title + "_tmp.mp3"
with open (audio_tmp_file, mode = "wb") as f :
    f.write(audio_content)
print("音频缓存成功！")

# 删除不需要的mp3
#os.remove(audio_tmp_file)

# # 转换mp3格式
mp3_file = "./data/" + title + ".mp3"
cmd = f"ffmpeg -i {audio_tmp_file} -vn -c:a libmp3lame -q:a 4 {mp3_file}"
print(cmd)
subprocess.run(cmd,shell=True)

print("音频转换成功！")

返回200，则网页请求成功： <Response [200]>
视频标题为： 地球的半径和质量都是怎么测量出来的？李永乐老师五分钟让你明白
音频地址为： https://cn-sxty-cu-03-01.bilivideo.com/upgcxcode/96/62/115346296/115346296_nb2-1-30280.m4s?e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M=&uipk=5&nbs=1&deadline=1698757264&gen=playurlv2&os=bcache&oi=2071460090&trid=0000d68990c05fa4439d97380c53bcd56ba1u&mid=0&platform=pc&upsig=02e27c00ba164c12875f18befda44989&uparams=e,uipk,nbs,deadline,gen,os,oi,trid,mid,platform&cdnid=71401&bvc=vod&nettype=0&orderid=0,3&buvid=&build=0&f=u_0_0&agrr=0&bw=22369&logo=80000000
音频缓存成功！
ffmpeg -i ./data/地球的半径和质量都是怎么测量出来的？李永乐老师五分钟让你明白_tmp.mp3 -vn -c:a libmp3lame -q:a 4 ./data/地球的半径和质量都是怎么测量出来的？李永乐老师五分钟让你明白.mp3
音频转换成功！


In [3]:
# 长音频分割成短音频
from pydub import AudioSegment

long_file = AudioSegment.from_mp3(mp3_file)

# PyDub handles time in milliseconds
ten_minutes = 15 * 60 * 1000 # 15分钟

total_length = len(long_file)

start = 0
index = 0
while start < total_length:
    end = start + ten_minutes
    if end < total_length:
        chunk = long_file[start:end]
    else:
        chunk = long_file[start:]
    with open(f"./data/{title}_{index}.mp3", "wb") as f:
        chunk.export(f, format="mp3")
    start = end
    index += 1

In [4]:
# 用whisper进行语音识别，注意Prompt增加中英文混排格式和标点符号

import openai, os

openai.api_key = os.getenv("OPENAI_API_KEY")

original_prompt =title + "。"
prompt = original_prompt
for i in range(index):
    print("index: ", i)
    clip = f"./data/{title}_{i}.mp3"
    audio_file= open(clip, "rb")
    print("audio file: ", clip)
    
    transcript = openai.Audio.transcribe("whisper-1", audio_file, response_format="srt",
                                     prompt=prompt)

    print(transcript)
    # mkdir ./data/srt if not exists
    if not os.path.exists("./data/srt"):
        os.makedirs("./data/srt")
    # write to file
    with open(f"./data/srt/{title}_{i}.srt", "w") as f:
        f.write(transcript)
    # get last sentence of the transcript
    sentences = transcript.split("。")
    prompt = original_prompt + sentences[-1]
    

NameError: name 'index' is not defined

In [1]:
# 用ChatGPT总结
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import SpacyTextSplitter
from llama_index import GPTListIndex, LLMPredictor, ServiceContext, SimpleDirectoryReader
from llama_index import Prompt
from llama_index.node_parser import SimpleNodeParser

# define LLM
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=1024))

text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 4096)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('./data/srt').load_data()
nodes = parser.get_nodes_from_documents(documents)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

list_index = GPTListIndex(nodes=nodes, service_context=service_context)
# tree_summarize （总结摘要-最优）
# simple_summarize （最省token）
# refine （基于关键词询问-最优）
# compact（较省token） 
query_engine = list_index.as_query_engine(response_mode="tree_summarize") 

response = query_engine.query("请你总结一下内容，要点限定在5条以内，并以Markdown格式显示")

print(response)

- 牛顿的万有引力定律描述了任何两个物体之间相互吸引的力，其大小由GMemr除以r的平方决定。
- 万有引力常数G是卡文迪许测量出来的，其值为6.67×10⁻¹¹牛米的平方每千克的平方。
- 卡文迪许通过实验测量出了万有引力常数G，他被称为测出地球质量的人。
- 人在地球上站立时受到的重力等于mg，其中g是重力加速度，约等于9.8牛每千克。
- 通过重力加速度g、地球质量和地球半径的关系，可以计算出地球的质量。
